# Expected Completion

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist #function to create adjacency matrix
import math
import tensorflow
import spektral
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dropout,Dense
from keras.callbacks import Callback, EarlyStopping

In [3]:
plays = pd.read_csv("data/pass_receiver.csv")
pff = pd.read_csv("data/pffScoutingData.csv")
games = pd.read_csv("data/games.csv")

read in weekly tracking files and concatenate into one tracking file, then delete weeklies

In [4]:
week1 = pd.read_csv("data/week1.csv")
week2 = pd.read_csv("data/week2.csv")
week3 = pd.read_csv("data/week3.csv")
week4 = pd.read_csv("data/week4.csv")
week5 = pd.read_csv("data/week5.csv")
week6 = pd.read_csv("data/week6.csv")
week7 = pd.read_csv("data/week7.csv")
week8 = pd.read_csv("data/week8.csv")
tracking = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])


In [5]:
#save memory
del week1,week2,week3,week4,week5,week6,week7,week8

In [6]:
plays = plays.merge(games[["gameId","homeTeamAbbr"]], on = "gameId")
plays["possTeamScoreDiff"] = np.where(plays["homeTeamAbbr"]==plays["possessionTeam"],plays["preSnapHomeScore"]-plays["preSnapVisitorScore"],plays["preSnapVisitorScore"]-plays["preSnapHomeScore"])
plays["possTeamInLead"] = np.where(plays["possTeamScoreDiff"]>0,1,0)

add unique identifier of each play and frame (some playIds repeat over games)

In [7]:
tracking["comb_id"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")
pff["comb_id"] = pff["gameId"].astype("str")+" "+pff["playId"].astype("str")
plays["comb_id"] = plays["gameId"].astype("str")+" "+plays["playId"].astype("str")
tracking["comb_and_frame"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")+" "+tracking["frameId"].astype("str")

isolate frames at time of pass

In [8]:
#isolate just forward passes and leave out a certain frame (duplicate forward pass for a play)
just_forward_passes = tracking[(tracking["event"]=="pass_forward") & (tracking["comb_and_frame"]!="2021091203 3740 32")& (tracking["team"]!="football")]
#bring in receiver info from plays
tracking_plays =just_forward_passes.merge(plays[["comb_id","possessionTeam","is_pass_failed","PassReceiver_nflId","down","yardsToGo","yardlineNumber","possTeamScoreDiff","possTeamInLead"]], how="left",on="comb_id" )
#and qb info from pff
tracking_merged_all = tracking_plays.merge(pff[["comb_id","nflId","pff_role"]], how = "left",on=["comb_id","nflId"])

isolate tracking to just QB, OL, receiver and defense

In [9]:
#tracking_players_filtered = tracking_merged_all[(tracking_merged_all["nflId"]==tracking_merged_all["PassReceiver_nflId"])|(tracking_merged_all["pff_role"]=="Pass")|(tracking_merged_all["pff_role"]=="Pass Block")|(tracking_merged_all["team"]!=tracking_merged_all["possessionTeam"])]
#tracking_players_filtered = tracking_merged_all[(tracking_merged_all["nflId"]==tracking_merged_all["PassReceiver_nflId"])|(tracking_merged_all["pff_role"]=="Pass")|(tracking_merged_all["team"]!=tracking_merged_all["possessionTeam"])]

# Function to take in 1 pass play and  format it

In [55]:
def one_play_function(data):
    one_play_formatted_list = pd.DataFrame()#{"y","Team","QB_Dist","DB_Dist","Orientation","Direction","Speed","down","yardsToGo","yardLine","ScoreDiff","inLead"})
    data = data.reset_index(drop=True)
    comb_id = data["comb_id"][0]
    #get y, the response variable for this play (binary pass completed or not) and possesionteam (for CV)
    y = data["is_pass_failed"][0]
    team = data["possessionTeam"][0]
    QB = data[data["pff_role"]=="Pass"]
    WR = data[data["nflId"]==data["PassReceiver_nflId"]]
    #if no clear receiver just pick anyone
    if len(WR)==0:
        WR = data[data["pff_role"]=="Pass Route"].head(1)
 
    qb_loc = QB[["x","y"]]
    wr_loc = WR[["x","y"]]
    coverage_loc = data[["x","y"]][data["pff_role"]=="Coverage"]
    #put play in adjacency matrix format, the distance of all players to each other (this will then be filtered to just link relevant players to QB or WR)
    QB_dist = cdist(qb_loc.values,wr_loc.values)[0]
    Coverage_dist = min((i for i in cdist(coverage_loc.values,wr_loc.values)[0] if i is not None),default=[])
    
    #then put play in features format, getting a table containing position, speed, orientation etc
    rel_o = WR["o"].iloc[0]-QB["o"].iloc[0]
    rel_dir = WR["dir"].iloc[0]-QB["dir"].iloc[0]
    s = WR["s"]
    one_play = pd.DataFrame({"comb_id":comb_id,"y":y,"Team":team,"QB_Dist":QB_dist,"DB_Dist":Coverage_dist,"Orientation":rel_o,"Direction":rel_dir,"Speed":s,"down":WR["down"],"yardsToGo":WR["yardsToGo"],"yardLine":WR["yardlineNumber"],"ScoreDiff":WR["possTeamScoreDiff"],"inLead":WR["possTeamInLead"]})
    #data = data[["onPossessionTeam","frameId","s","rel_dir","rel_o","pff_role","down","yardsToGo","yardlineNumber","possTeamScoreDiff","possTeamInLead"]]
    
    one_play_formatted_list = pd.concat([one_play_formatted_list,one_play], ignore_index = True)
    return one_play_formatted_list

In [56]:
table_list = pd.DataFrame()
for i in range(len(tracking_merged_all.comb_id.unique())):
#for i in range(10):
    
    one_play = tracking_merged_all[tracking_merged_all.comb_id==tracking_merged_all.comb_id.unique()[i]]
    #print(tracking_players_filtered.comb_id.unique()[i])
    one_play = one_play.reset_index(drop=True)
    result = one_play_function(one_play)
    table_list = pd.concat([table_list,result],ignore_index=True)


In [42]:
range(len(tracking_merged_all.comb_id.unique()))


range(0, 7547)

In [57]:
len(table_list)

7558

In [15]:
np.array(table_list.loc[:, "y"])

array([1, 0, 0, ..., 0, 0, 1], dtype=int64)

get list of indexes of possession teams to allow masking for Cross Validation

# NN Functions

In [28]:
#variables
learning_rate = 1e-3  # Learning rate
epochs = 100  # Number of training epochs
es_patience = 10 # Patience for early stopping
batch_size = 32  # Batch size
callbacks = [EarlyStopping(monitor='val_binary_accuracy', patience=10, verbose=0)]
unique_team_list = table_list.Team.unique()

train xCompletion model

In [26]:
# Define the model
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=9))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['binary_accuracy'])



Cross Validation with k = 32 by loop through and for each team use as val (remove from training) and store predictions

In [29]:
prediction_list = []
for i in range(len(unique_team_list)):
    train = table_list[table_list["Team"]!=unique_team_list[i]]
    val = table_list[table_list["Team"]==unique_team_list[i]]
    x_train = np.array(train.loc[:, "DB_Dist":"inLead"])
    y_train = np.array(train.loc[:,"y"])
    x_val = np.array(val.loc[:, "DB_Dist":"inLead"])
    y_val = np.array(val.loc[:,"y"])
    model.fit(x_train, y_train, validation_data=(x_val, y_val), 
                    epochs=epochs, batch_size=batch_size, callbacks = [callbacks])
    predictions = model.predict(np.array(table_list.loc[:, "DB_Dist":"inLead"]))
    prediction_list.append(predictions)
    

Epoch 1/100
228/228 [==============================] - 0s 2ms/step - loss: 0.3661 - binary_accuracy: 0.6249 - val_loss: 0.3690 - val_binary_accuracy: 0.6254
Epoch 2/100
228/228 [==============================] - 0s 2ms/step - loss: 0.3632 - binary_accuracy: 0.6293 - val_loss: 0.3776 - val_binary_accuracy: 0.6117
Epoch 3/100
228/228 [==============================] - 0s 1ms/step - loss: 0.3679 - binary_accuracy: 0.6206 - val_loss: 0.3783 - val_binary_accuracy: 0.6220
Epoch 4/100
228/228 [==============================] - 0s 1ms/step - loss: 0.3594 - binary_accuracy: 0.6282 - val_loss: 0.3686 - val_binary_accuracy: 0.6220
Epoch 5/100
228/228 [==============================] - 0s 2ms/step - loss: 0.3560 - binary_accuracy: 0.6245 - val_loss: 0.3636 - val_binary_accuracy: 0.6186
Epoch 6/100
228/228 [==============================] - 0s 1ms/step - loss: 0.3411 - binary_accuracy: 0.6223 - val_loss: 0.3248 - val_binary_accuracy: 0.5773
Epoch 7/100
228/228 [==============================] - 0s 

In [30]:
model_output = np.mean(prediction_list, axis=0)

In [40]:
len(table_list)

7558

In [41]:
tracking_merged_all.comb_id.nunique()

7547

In [34]:
#pass_forwards = tracking[["comb_id"]][(tracking["event"]=="autoevent_pass_forward")|(tracking["event"]=="pass_forward")].drop_duplicates()#.groupby("comb_id").min()
play_with_pass = plays[plays["comb_id"].isin(tracking_merged_all.comb_id)]
play_with_pass = play_with_pass.drop_duplicates(subset="comb_id")
play_with_pass["pred"] = model_output.tolist()
play_with_pass.to_csv("prediction_output.csv")

ValueError: Length of values (7558) does not match length of index (7547)

In [47]:
import xgboost as xgb

# Define the XGBoost model
model = xgb.XGBClassifier()

prediction_list = []
for i in range(len(unique_team_list)):
    train = table_list[table_list["Team"]!=unique_team_list[i]]
    val = table_list[table_list["Team"]==unique_team_list[i]]
    x_train = np.array(train.loc[:, "DB_Dist":"inLead"])
    y_train = np.array(train.loc[:,"y"])
    x_val = np.array(val.loc[:, "DB_Dist":"inLead"])
    y_val = np.array(val.loc[:,"y"])
    model.fit(x_train, y_train)
    predictions = model.predict(np.array(table_list.loc[:, "DB_Dist":"inLead"]))
    prediction_list.append(predictions)

In [48]:
model_output2 = np.mean(prediction_list, axis=0)

In [54]:
len(model_output2)

7558

In [58]:
table_list2 = table_list.copy()
table_list2["pred"] = model_output2.tolist()
table_list2.to_csv("prediction_output.csv")

In [60]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics

mean_squared_error(table_list2["y"], table_list2["pred"])

0.07283569562053453

In [67]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_test = table_list2["y"]
y_pred = np.where(table_list2["pred"]>.5,1,0)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='binary')

In [68]:
print(accuracy)

0.9001058481079651


In [70]:
print(precision)

0.9635747564591275


In [71]:
print(recall)

0.7727581521739131


In [69]:
print(f1)

0.8576814326107446
